## Plan for continuous training

Here's the plan. The script boots up and checks a continuous play directory which is populated with a number of training round sub-directores. Each sub-directory is labelled in sequence and contains a self_play sub-sub-directory. The script takes stock of the contents of the latest training round self_play directory.

If the number of files (tournaments) saved in the self_play directory of the latest training round is less than some minimum M, say m < M then the self-play script is kicked off to play M - m more tournaments, completing the M requred self-play tournaments and saving to the  subdirectory.

If the number of files saved in the self_play directory of the latest training round m >= M then the script checks if there is already a model saved in the training round directory.

If there is not a model saved in the training round directory, the script will kick off a training routine to create a new model (or load a saved model from a possibly existing previous round), compile a training dataset from the most recent k training rounds (including all known checkmate positions) and train the model to stopping. The script will then save the model in the training round sub-directory.

Then the script will create a new training round directory, labelled with the next integer in the training round sequence, containing a self_play directory. At this point we can essentially continue the loop from the top.

Turns out multiprocessing is an excellent solution to CUDA memory management. Spinning up a separate process for training means all memory allocation is removed afterward.

In [1]:
import os, torch
from chess_selfplay import harvest_checkmates
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

min_tournaments_each_round = 50 # A new model roughly once a day.
model_kwargs = {'nlayers':6, 'nheads':3, 'embed_dim':12, 'dk':8, 'device':device}
root_dir = os.path.join('data','output')
dir_sort_key = lambda d: int(d.split('_')[-1])
mode = 'continuous'

## LOOP STARTS - just kill the machine any old time when you have stuff to do and you can fire it up again whenever you're ready.
while True:

    # Checking where we're up to
    training_round_dirs = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir,d))], key=dir_sort_key)
    current_training_round_dir = training_round_dirs[-1]
    previous_training_round_dir = training_round_dirs[-2]

    # Counting how many tournaments have been played already this round
    current_self_play_path = os.path.join(root_dir, current_training_round_dir, 'self_play')
    if os.path.exists(current_self_play_path) and os.path.isdir(current_self_play_path):
        number_games_played_already = len(os.listdir(current_self_play_path))
    else:
        os.mkdir(current_self_play_path)
        number_games_played_already = 0

    # Play more tournaments if not enough in latest batch
    tournaments_left_to_play = min_tournaments_each_round - number_games_played_already
    print(f'Current: {current_training_round_dir}, already played: {number_games_played_already}, left to play: {tournaments_left_to_play}')
    if tournaments_left_to_play > 0:
        
        # self-play script to play tournaments_left_to_play more tournaments, saving to current_self_play_dir.
        previous_model_path = os.path.join(root_dir, previous_training_round_dir, 'model.pt')
        assert os.path.exists(previous_model_path) and os.path.isfile(previous_model_path), 'ERROR: MODEL NOT FOUND.'

        # Same base model with different look-ahead strength configuration 
        model_kwargs['load_path'] = previous_model_path
        agent0_spec = {'type':'transformer', 'kwargs':model_kwargs, 'num_simgames':150, 'max_simmoves':6, 'C':1, 'p':0.3, 'k':float('inf')}
        agent1_spec = {'type':'transformer', 'kwargs':model_kwargs, 'num_simgames':1,   'max_simmoves':1, 'C':1, 'p':0.4, 'k':float('inf')}
        selfplay_args = {
            'num_workers':2, 'num_tournaments': tournaments_left_to_play, 'agents_spec': [agent0_spec, agent1_spec], 
            'num_games':1, 'starting_state':None, 'max_moves':200, 'save':True, 'result_dest':current_self_play_path
        }

        # Let's play
        print(f'Playing {tournaments_left_to_play} tournaments...')
        %run -i "chess_selfplay.py"
        print(f'Self-play complete.')

    # Extract the checkmates from the current_self_play_dir tournament games and save them in the current_training_round_dir.
    current_round_checkmates_path = os.path.join(root_dir, current_training_round_dir, 'checkmates.pkl')
    if not (os.path.exists(current_round_checkmates_path) and os.path.isfile(current_round_checkmates_path)):
        _ = harvest_checkmates(os.path.join(root_dir, current_training_round_dir))

    # If there's not already a model saved in the current training directory, kick off a training run
    latest_model_path = os.path.join(root_dir, current_training_round_dir, 'model.pt')
    if not (os.path.exists(latest_model_path) and os.path.isfile(latest_model_path)):

        # Namespace for chess_modeltraining.py includes:
        print(f'Training using play from {previous_training_round_dir}, testing on {current_training_round_dir}.')
        modeltraining_args = {
            'mode': mode,
            'root_dir': root_dir, 
            'previous_training_round_dir': previous_training_round_dir,
            'current_training_round_dir': current_training_round_dir, 
            'model_kwargs': model_kwargs, 
            'device': device
        }
        # Let's train
        %run -i "chess_training.py"
        print(f'Training complete.')
        
    # Create next training round directory containing self_play sub-directory, and start loop from the top.
    next_index = int(os.path.split(current_training_round_dir)[-1].split('_')[-1]) + 1
    next_training_round_dir = f'round_{next_index}'
    print(f'Creating next training round directory {next_training_round_dir}')
    next_training_round_path = os.path.join(root_dir, next_training_round_dir)
    os.mkdir(next_training_round_path)
    next_training_round_self_play_path = os.path.join(next_training_round_path, 'self_play')
    os.mkdir(next_training_round_self_play_path)

cuda
Current: round_7, already played: 33, left to play: 17
Playing 17 tournaments...
Self-play complete.
Training using last self-play as test set...
Training complete.
Creating next training round directory round_8
Current: round_8, already played: 0, left to play: 50
Playing 50 tournaments...
